## Mutable variables

In [ ]:
var i:Int = 1

We can reassign a value to `i`

In [ ]:
i = 2

## Immutable values

In [ ]:
val s:String = "a"

Declaring `s` as `val` prevents new assignement

In [ ]:
s = "c"

## Function

We can declare inline function, the type will take a form of an application, as we can see below where the function applies a `Int` onto a `String`.

In [ ]:
val f:String => Int = 
  (s: String) => {
    val removeWhiteChars = s.replaceAll(" ", "")
    s.size
  }

In [ ]:
f("So, Scala is as powerful as it is simple !")

Since functions can be defined as `val`s then they can be _serialized_. This is **crucial** to understand since it is linked to closures serialization.

> Note: the `return` can be avoided because every thing is an expression, see below

## Types

Scala is strongly and statically typed, hence we can encapsulate our structure (or even behavior) using a powerful object oriented system.

### Class

In [ ]:
class NewClass(val a:String, b:Int)

Instances of `NewClass` can be created using `new`.

In [ ]:
val newClassInstance = new NewClass("ok", 1)

`a` is a public field

In [ ]:
newClassInstance.a

`b` is a constructor parameter

In [ ]:
newClassInstance.b

#### Methods

The behavior can be declared using methods (local functions), a method is essentially like this:

```
def <method name> ( (<arg name> : <type name>)* ) = <body>
```

So we assign (`=`) a body to a name with its typed arguments.

In [ ]:
class ClassWithBehavior(i:Int) {
  def add(j:Int) = i + j
  def random():Int = i + scala.util.Random.nextInt
}

In [ ]:
val classWithBehaviorInstance = new ClassWithBehavior(5)

In [ ]:
classWithBehaviorInstance.add(10)

In [ ]:
classWithBehaviorInstance.random()

####  `apply`

There is a special method in Scala that has an important meaning: `apply`.

If an `apply` method is declared in any type (see below) then an instance of this type can be used as a function which respects the `apply` signature.

In [ ]:
class ThisCanBeAFunction(log:String=>Unit) {
  def apply(s:String) = log(s)
}

In [ ]:
val trickyFun:ThisCanBeAFunction = new ThisCanBeAFunction((s:String) => println("trick: " + s))

In [ ]:
trickyFun("no apply")

> Note: `Unit` is like void like in any other language, but it has a type in Scala and can be used as such. Its only instance is the unit value `()`.

### Trait

Abstraction in Scala can be declared into `trait`s which are like interfaces but allow more flexibility: a trait can be mixed in, have a state, implement methods.

However, a trait cannot be instantiated and needs to be extended first.

In [ ]:
trait Partial {
  def todo:Double
  
  val value:Int = 10
  
  def done:String = ""+scala.util.Random.nextPrintableChar
}

In [ ]:
class FullType extends Partial {
  override val value:Int = 20
  
  def todo:Double = scala.util.Random.nextDouble
}

### Object

Sometimes we just want to lift an implementation as an object (thing about singleton). For this, you can simply declare a new structure to be an `object` which is combining declaration and instantiation.

In [ ]:
object FullObject extends Partial {
  val todo:Double = 1.0
}

In [ ]:
FullObject.todo

### Type Parameter (~ generic)

Some behavior can be declared without fixing the real values onto which it'll be applied, in Java this is called generics.

Scala allows type parameters to be associated to high level types in order to bag behaviors together (for instance).

In [ ]:
trait Converted[I, O] {
  def i:I
  def convert(i:I):O
  val o:O = convert(i)
}

We can now override and give _type values_.

Note: we can also override the modifier for field, `def` can be overriden with a `val`.

In [ ]:
class StringToInt(val i:String) extends Converted[String, Int] {
  def convert(s:String) = s.toInt
}

In [ ]:
val converted:StringToInt = new StringToInt("124")

In [ ]:
val convertedO:Int = converted.o

## Case

The language keyword `case` can be used in different scenarii but all are about **dealing with structures that can be parsed and introspected safely**. This statement implies that the structure has to be static and known by the compiler.

### Immutable structures

Based on the fact that the structure is statically known at compile time, the compiler can help on several phases. One thing he can do for instance is to generate code based on the known structure. For that purpose, the `case` modifier is used to declare immutable structures. That is because immutable structures are hard to deal for reasons like updating the values requires a copy constructor which is hard to maintain.

Immutable structures in Scala are then defined using `case` and will generate (at least):
* an attached object with **the same name**, called the companion object, defining a static `apply` function that acts as a factory method
* copy constructor: which create a new instance based on all untouched values of the current instance identical
* deconstructor: which explodes the current structure in a way that we can deal with internal values (see **Pattern Matching** below).

In [ ]:
case class ImmutableClass[V](k:String, value:V)

We can now create an instance of this class.

The `new` keyword can be omitted because the function `ImmutableClass.apply` is actually called.

In [ ]:
val immutableData:ImmutableClass[Int] = ImmutableClass[Int]("zero", 0)

The `copy` method is now available

In [ ]:
immutableData.copy("o", immutableData.value)

#### Default and named values

Using the `copy` function (for instance) can be tedious if there are many arguments, also you don't want to repeat yourself.

So Scala has also the concept of default values for function's arguments, in the case of `copy` the Scala compiler has assigned the value of the current instance as the default value for each respective argument.

To simplify its usage (regarding order specially), there is also the concept of named parameter, allowing the arguments to be given a value tagged with their name

In [ ]:
case class ImmutableClassWithDefault[V](k:String, value:V)

In [ ]:
val immutableClassWithDefaultInstance:ImmutableClassWithDefault[Double] = ImmutableClassWithDefault[Double]("zero", 0.0)

immutableClassWithDefaultInstance.copy(k = "o")

### Interesting Types

#### Tuples

In [ ]:
val stringAndFloat:(String, Float) = ("one", 1)
val stringAndFloatAndInt:(String, Float, Int) = ("one", 1, 1)

We can then access safely element by index, yet be able to rely on the type infomation.

In [ ]:
stringAndFloatAndInt._2 + " is a Float"

In [ ]:
:markdown

**Note**: string interpolation is also possible like this

${stringAndFloatAndInt._2} is a Float

#### Option

Null are bad, Scala hates `null`s.

Since it's still relevant to have _no values_ for an object, and we want to deal with it safely, Scala has used a (now) common pattern: using optional values.

An optional value is a type that has only two forms:
* `None`: no value
* `Some(v)`: has a value `v`

Option has many advantages and methods attached to it, although it can seem to complexify the code, with a bit of practice `Option` becomes a killer feature that increase the code, yet increasing its safety, some interesting functions are listed below.

In [ ]:
val noValueOption:None.type = None
val someStringValue :Some[String] = Some("ok")
val optionString1:Option[String] = noValueOption
val optionString2:Option[String] = someStringValue

> Note: `Nothing` is a specific type in Scala which is actually **deriving all** types, hence it is always in the bottom of the type hierarchy and can be assigned to any other.

In [ ]:
:markdown

- `getOrElse`: return the value or the provided default: **${noValueOption.getOrElse("hello")}** and **${someStringValue.getOrElse("no")}**
- `map`: deal with contained value if any: **${optionString2.map((s:String) => s.size)}**
- `orElse`: chain optional values: **${optionString1 orElse optionString2}**

> Note: sometimes omitting the `.` in Scala can make sense like in `optionString1 orElse optionString2`.

> Note': so, yes dots and parenthesis can be omitted in Scala

#### Collections

Collections in Scala are by default immutable structures from the package `scala.collection.immutable`, so you can use them like any other `case class` for instance.

In [ ]:
val oneToThree = List(1, 2, 3)

Hence adding an element needs an extra object referring to the previous one as the tail or the init (for instance):

In [ ]:
val oneToFour = oneToThree :+ 4

The initial list remains though

In [ ]:
oneToThree

### Pattern matching

One of the most interesting feature of the Scala language regarding conciseness and readability is its pattern matching.

Thanks to static typing, immutable values and other characteristics of the language we can safely introspect structures and values to deal with different logical paths.

This is the second usage of `case`, it will be now used to declare a structure that need to be matched by a given value. When a match is found, the behavior attached to the case is executed.

Think of it as a powerful `switch` statement.

In [ ]:
(optionString1 orElse optionString2) match {
  case Some("ok") => 0
  case Some(x) if x.size >2 => 1
  case x => // small leter == catch all
    val message = "unknown"
    s"$x is $message"
}

Match are typesafe, and thus `case` can be checked, so that you'll avoid mixing things up:

In [ ]:
(optionString1 orElse optionString2) match {
  case Some("ok") => 0
  case Some(x) if x.size >2 => 1
  
  case "not a string" => ???
  
  case x =>
    val message = "unknown"
    s"$x is $message"
}

## Everything is an expression

Whatever you do in Scala, you will return a value. This value will be simply the last result of the last statement of the expression (code block, ...).

This avoid decoupling the declaration with the assignement like

```
var s:String = null
if (true) {
  s = "ok"
} else {
  s = "nok"
}
``` 

### `if`

In [ ]:
val ifExp:String = if (true) "ok" else "nok"

### `for`

In [ ]:
val forExp:Seq[(Int, Int)] = for {
  listElement1 <- 1 to 10
  listElement2 <- 2 to 20 by 2
} yield (listElement1, listElement2)

## Types are inferred

By the way, if you felt submerged by type notations... I agree. Luckily, although Scala is strongly and statically typed, type tagging is optional.

Since everything is an expression, everything has a type and the type can be inferred by the last statement of the expression. The upper type in the common hierarchy is taken if there are several branches in the expressions that need to be resolved. The highest type being `Any`.

In [ ]:
val optionString = if (scala.util.Random.nextBoolean) None else Some("ok")

In [ ]:
val listOfDoubles = optionString match {
  case Some("ok") => List(1.0, 2, 3)
  case Some(x) => Nil
  case None => List(0d)
}

In [ ]:
val mess = listOfDoubles.map(x => (x, scala.util.Random.nextBoolean)).map { 
  case (x, true) => s"$x is true"
  case (x, false) if x > 2 => x - 2
  case x => x
}

In [ ]:
import scala.concurrent.Future
import scala.concurrent.ExecutionContext.Implicits.global
val complex = Future { Left ( Option ( List( (1, 'a'), (2, 'b') ) zip List("one", "two") ) ) }

## Short notation: `_`

When using functional paradigm like in Spark for instance, we have to deal with a ton of functions in order to describe the logic in a smooth and comprehensible manner.

Which means that we regularly have to define small and sometimes trivial functions, which look like bloated using the full function definition notation.

In [ ]:
List(1, 2, 3).
  map(x => x + 1).
  map(x => (x, x % 2)).
  toMap.
  mapValues(x => x == 0)

The above code can take a simple form using the short notation, using the underscore (`_`) as a replacement for the $i^{th}$ argument.

In [ ]:
def addMethod(x:Int, y:Int):Int = x + y

val add = (x:Int, y:Int) => x + y

val addShort:(Int, Int)=>Int = _ + _

In [ ]:
val listOfTuples = (1 to 100)

listOfTuples.reduce(addMethod)

listOfTuples.reduce(add)

listOfTuples.reduce(addShort)

listOfTuples.reduce((x:Int, y:Int) => x + y)

listOfTuples.reduce(_ + _)